# PRATICA_INDEPENDENTE - Pandas 2.

## Introdução

### Contexto:

#### Esse conjunto de dados é um registro de cada edifício ou unidade de edifício (apartamento etc.) vendidos no mercado imobiliário de Nova York durante um período de 12 meses.

### Conteúdo:

#### Esse dataset contem o local (location), endereço (address), tipo (type), preço de venda (sale price) e data de venda (sale date) de unidades do edifício. Veja a seguir algumas referências  sobre os campos:

* BOROUGH: Um código para definir o bairo em que a propriedade está localizada:
    - Manhattan (1), 
    - Bronx (2), 
    - Brooklyn (3), 
    - Queens (4), 
    - Staten Island (5).

* BLOCK; LOT: A combinação do bairro "borough", bloco "block", e lote "lot" forma uma chave única para a propriedade em New York City. Chamado de BBL.

* BUILDING CLASS AT PRESENT e BUILDING CLASS AT TIME OF SALE: O tipo de edifício em vários pontos no tempo. Veja o glossário abaixo:

#### Para referência adicional em campos individuais, consulte o [Glossário de Termos](https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf). Para os códigos de classificação de construção, consulte o Glossário de classificações de construção de [NYC Property Sales](https://www.kaggle.com/new-york-city/nyc-property-sales).

## Importamos os pacotes necessários e carregamos os dados.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

In [19]:
dataset = pd.read_csv('nyc-rolling-sales_twentieth.csv')

### Exercício 1: Avalie os [tipos](https://realpython.com/python-data-types/#type-conversion) das colunas e faça as alterações necessárias.

In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16909 entries, 0 to 16908
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Unnamed: 0                      16909 non-null  int64         
 1   BOROUGH                         16909 non-null  int64         
 2   NEIGHBORHOOD                    16909 non-null  object        
 3   BUILDING CLASS CATEGORY         16909 non-null  object        
 4   TAX CLASS AT PRESENT            16909 non-null  object        
 5   BLOCK                           16909 non-null  int64         
 6   LOT                             16909 non-null  int64         
 7   EASE-MENT                       16909 non-null  object        
 8   BUILDING CLASS AT PRESENT       16909 non-null  object        
 9   ADDRESS                         16909 non-null  object        
 10  APARTMENT NUMBER                16909 non-null  object        
 11  ZI

### Vamos aterar os tipos de algumas colunas.

In [20]:
dataset['SALE DATE'] = pd.to_datetime(dataset['SALE DATE'])

### Vamos eliminar as linhas que contêm valores `NaN`.

In [15]:
## Como podemos ver no '.info()', nesta database não há nulos, mes se tivesse
dataset.dropna(inplace=True)

### Qual é o valor médio do metro quadrado em NY?

In [42]:
# dataset['SALE PRICE']
def w(i):
    if i == ' -  ':
        return 0
    else:
        return i
    
dataset['SALE PRICE'] = dataset['SALE PRICE'].apply(w)

# dataset[dataset['SALE PRICE'].index == 2]
# dataset['SALE PRICE'][1] Especifica item

dataset['SALE PRICE'] = pd.to_numeric(dataset['SALE PRICE'])

In [64]:
# dataset['GROSS SQUARE FEET']

def zx(i):
    if i == ' -  ':
        return 0
    else:
        return int(i)
    
dataset['GROSS SQUARE FEET'] = dataset['GROSS SQUARE FEET'].apply(zx)

dataset['GROSS SQUARE FEET'] = pd.to_numeric(dataset['GROSS SQUARE FEET'])

# dataset['GROSS SQUARE FEET'][16908]

In [73]:
result = dataset['SALE PRICE'].mean() / dataset['GROSS SQUARE FEET'].mean()
round(result,2)

774.7

### Qual o preço médio por metro quadrado de cada `BLOCK`? Organizar os dados para indicar qual é o mais caro.
Nota: fazer o cálculo tanto com groupby como com pivot tables

### Em qual `BLOCK` há maior dispersão de preços por metro quadrado? Organizar os valores para identificar o maior.

(Lembrar da fórmula do coeficiente de variação para medir a dispersão)

**Pistas**
* A primeira opção é definir uma função com arrays e utilizar `.apply()`

* A segunda, é gerar duas séries: 
  - uma com o método `.std()` e dividi-la por outra série gerada com `mean()`

### Em qual bairro os apartamentos são maiores? 
Nota: o cálculo pode ser feiro tanto com groupby como com pivot tables

In [81]:
def zx(i):
    if i == ' -  ':
        return 0
    else:
        return int(i)
    
dataset['LAND SQUARE FEET'] = dataset['LAND SQUARE FEET'].apply(zx)
dataset['LAND SQUARE FEET'] = pd.to_numeric(dataset['LAND SQUARE FEET'])

In [82]:
dataset[['NEIGHBORHOOD','LAND SQUARE FEET']].groupby('NEIGHBORHOOD').max()\
                        .sort_values(by='LAND SQUARE FEET',ascending=False)

,LAND SQUARE FEET
NEIGHBORHOOD,
FASHION,364000
SOUTHBRIDGE,357935
HARLEM-CENTRAL,189025
KIPS BAY,141836
TRIBECA,94530
MIDTOWN CBD,81336
SOHO,69096
FINANCIAL,53632
UPPER WEST SIDE (79-96),40350


### Em geral, você pode ver alguma diferença entre o preço médio por metro quadrado dos apartamentos, considerando seu ano de construção? o que você pode dizer sobre a relação entre o ano de construção e o tamanho total médio dos mesmos em pés quadrados?

### Gere um `DataFrame` que acrescente a informação por (`PRICE PER SQUARED FEET`), unidades resideinciais (`RESIDENTIAL UNITS`) e unidades comerciais (`COMMERCIAL UNITS`) por `BLOCK` e vizinhança (`NEIGHBORHOOD`). Forneça informações sobre a tendência central e a dispersão de ambas as distribuições.